### Trabajo Práctico 1: Modelos Predictivos en SciKit-Learn

- Paloma Monzon Borioli
- Camila Belen Vivo
- Martin Berestovoy

https://github.com/IgnacioPardo/Tecnologias_Exponenciales_2023/blob/main/Consigna_ModelosPredictivos.md

#### Avanzado Tecnologías Exponenciales 2023

#### Consigna

En grupos de 2 o 3 personas, realizar los siguientes ejercicios.



### Ej 1. 


Investigar y seleccionar un dataset que cumpla con tener entre 1000 y 10.000 muestras, 5 o más atributos numéricos y al menos un atributo categórico (Recomendación: seleccionar un atributo a predecir binario). De encontrar algún dataset sin atributos categóricos, ¿Como se podría generar uno binario a partir de los atributos numéricos? Se recomienda utilizar Kaggle para la búsqueda del dataset. Antes de avanzar con el trabajo práctico, corroborar el dataset en clase.



In [ ]:
import seaborn as sns
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
import warnings
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, RidgeClassifierCV, LassoCV
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, make_scorer, accuracy_score, classification_report

In [ ]:
# Importamos el dataset
df = pd.read_csv('athletes.csv')

# Eliminamos las filas con datos nulos
df = df.dropna()

# Eliminamos las columnas innecesarias
columns_to_drop = ['Games', 'Name', 'Season', 'City', 'Event', 'NOC', 'ID']
df = df.drop(columns=columns_to_drop)

# Filtramos el dataset para que solo tenga 10000 filas
df = df.sample(n=10000)


### Ej 2. 


Realizar un análisis exploratorio de los datos. Se recomienda utilizar gráficos para visualizar la distribución de los datos y la correlación entre los atributos. Se recomienda utilizar la librería `seaborn` para realizar los gráficos.




In [ ]:
# Desactivamos advertencias que son bastante molestas
warnings.simplefilter(action='ignore', category=FutureWarning)


# Distribucion de variables numericas

# Hacemos el grafico y definimos la separacion de los parametros con el bins
sns.histplot(data=df, x='Age', bins=50)
# Mostramos el grafico
plt.show()

sns.histplot(data=df, x='Height', bins=50)
plt.show()

sns.histplot(data=df, x='Weight', bins=50)
plt.show()

sns.histplot(data=df, x='Year', bins=50)
plt.show()


# Distribucion de variables categoricas


# Configuramos el tamaño de la figura
plt.figure(figsize=(20, 10))
sns.countplot(data=df, x='Sport')
# Definimos la rotacion de las palabras para que esten en vertical, mas chicas y en el centro asi son legibles
plt.xticks(rotation=90, fontsize=20, ha='center')
plt.show()

plt.figure(figsize=(10, 10))
sns.countplot(data=df, x='Medal')
plt.show()

plt.figure(figsize=(10, 10))
sns.countplot(data=df, x='Sex')
plt.show()

plt.figure(figsize=(50, 15))
sns.countplot(data=df, x='Team')
plt.xticks(rotation=90, fontsize=10, ha='center')
plt.show()
# Chona, te juramos que no se puede hacer mas legible el grafico por la abrupta cantidad de paises que tiene, sabe disculparnos :)

### Ej 3. 


Como resultado del análisis exploratorio, seleccionar un atributo categórico y un atributo numérico para realizar un modelo de clasificación. Se recomienda utilizar la función `LabelEncoder` de SciKit-Learn para convertir el atributo categórico a numérico.




In [ ]:
# Elegimos como atributo numerico la edad y como atributo categorico la medalla

# Creamos un diccionario de mapeo de valores
medal_mapping = {"Gold": 1, "Silver": 2, "Bronze": 3}
# Aplicamos el mapeo a la columna "Medal"
df['Medal'] = df['Medal'].map(medal_mapping)

# Aca hacemos un encoding de las columnas que son categoricas
leTeam = preprocessing.LabelEncoder()
leSport = preprocessing.LabelEncoder()

leTeam.fit(df['Team'])
leSport.fit(df['Sport'])

df['Team'] = leTeam.transform(df['Team'])
df['Sport'] = leSport.transform(df['Sport'])

# Hacemos una columna booleana para ver si es hombre o no (mujer)
df['IsMale'] = df['Sex'].replace({'M': True, 'F': False})
df = df.drop(columns='Sex')

### Ej 4. 


Realizar una partición de los datos en conjuntos de entrenamiento y test. La proporción con la cual hacen esta partición es libre. Se recomienda utilizar la función `train_test_split` de SciKit-Learn.

In [ ]:
# Realizamos la particion de los datos en entrenamiento y testeo

# Dividimos el conjunto de datos en características (X) y la variable objetivo (y)
X_num = df.drop(columns=['Age'])  # Eliminamos la columna 'Age' como nuestra variable objetivo ya que en base a las otras columnas queremos predecir la edad
y_num = df['Age']

X_cat = df.drop(columns=['Medal'])  # Eliminamos la columna 'Medal' como nuestra variable objetivo ya que en base a las otras columnas queremos predecir la medalla
y_cat = df['Medal']

# Realizamos la particion de los datos en conjuntos de entrenamiento y prueba
X_train_num, X_test_num, y_train_num, y_test_num = train_test_split(X_num, y_num, test_size=0.2, random_state=42)
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_cat, y_cat, test_size=0.2, random_state=42)

### Ej 5. 


Para el atributo numérico a predecir seleccionado:

- Realizar un modelo de regresión lineal utilizando la clase `LinearRegression` de SciKit-Learn.
- Realizar un modelo de Árbol de Decisión utilizando la clase `DecisionTreeRegressor` de SciKit-Learn. Seleccionar hiperparámetros que les parezca mejoren el modelo.

Responder:
¿Que formas tienen de evaluar los resultados de cada árbol de decisión? ¿Como eligen "el mejor árbol"? ¿Como se comparan los resultados de los modelos de regresión lineal y de árbol de decisión?

Sugerencia: Aprovechar los conceptos de validación y de validación cruzada para evaluar los modelos. Pueden utilizar la función `cross_val_score` de SciKit-Learn para evaluar los modelos.

Se recomienda utilizar la función `cross_val_score` de SciKit-Learn para evaluar los modelos.




In [ ]:
# Definimos la cantidad de folds
k_folds = 5

# Evaluamos la Regresion Lineal
linear_regresion_scores = cross_val_score(LinearRegression(), X_num, y_num, cv = k_folds, scoring = make_scorer(mean_squared_error))

# Evaluamos el Arbol de Decision sin afinar hiperparametros
decision_tree_scores = cross_val_score(DecisionTreeRegressor(), X_num, y_num, cv = k_folds, scoring = make_scorer(mean_squared_error))

# Calculamos la media y desviacion estandar de los puntajes de MSE para los modelos
print("Regresion Lineal MSE:", round(linear_regresion_scores.mean(), 2)) # Ponemos ese round para que no salga con tantos decimales
print("Arbol de Decision MSE:", round(decision_tree_scores.mean(), 2))

# Cuanto mas chico sea el MSE, mejor es el modelo.
if(linear_regresion_scores.mean() < decision_tree_scores.mean()):
    print("El modelo de Regresion Lineal es mejor que el modelo de Arbol de Decision")
else:
    print("El modelo de Arbol de Decision es mejor que el modelo de Regresion Lineal")

# Afinamos hiperparametros del Arbol de Decision
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Realizamos la busqueda de hiperparametros con GridSearchCV y 5 folds de validacion cruzada (k_folds) y scoring de MSE (mean_squared_error) para el Arbol de Decision sin afinar hiperparametros (DecisionTreeRegressor()) y los hiperparametros definidos en el diccionario (param_grid)
grid_search = GridSearchCV(DecisionTreeRegressor(), param_grid, cv = k_folds, scoring = make_scorer(mean_squared_error))
grid_search.fit(X_num, y_num)

# Evaluamos el Arbol de Decision con hiperparametros optimizados
dt_optimized = DecisionTreeRegressor(**grid_search.best_params_)
dt_optimized_scores = cross_val_score(dt_optimized, X_num, y_num, cv = k_folds, scoring = make_scorer(mean_squared_error))

# Calculamos la media y desviacion estandar de los puntajes de MSE para el Arbol de Decision optimizado
print("Arbol de Decision optimizado MSE:", round(dt_optimized_scores.mean(), 2))

#### Respondemos a las preguntas:

##### ¿Que formas tienen de evaluar los resultados de cada árbol de decisión?
Los modelos se evaluan con un "score", es decir que se pone a prueba el modelo con un conjunto de datos de testeo y se obtiene un valor que indica que tan bien se comporta el modelo con esos datos.
##### ¿Como eligen "el mejor árbol"?
El mejor arbol es el que tiene el mejor score, es decir el que mejor se comporta con los datos de testeo. 
##### ¿Como se comparan los resultados de los modelos de regresión lineal y de árbol de decisión?
Se comparan viendo el score, el que tenga un score mas alto es el que mejor se comporta con los datos de testeo.

### Ej 6. 


Para el atributo categórico a predecir seleccionado

- Realizar un modelo de clasificación utilizando la clase `LogisticRegression` de SciKit-Learn.
- Realizar un modelo de clasificación utilizando la clase `DecisionTreeClassifier` de SciKit-Learn.

Responder las mismas preguntas que en el punto 5 para este caso.

In [ ]:
# Modelo de Regresion Logistica
logistic_regression_model = LogisticRegression()

# Mostramos el score del modelo de Regresion Logistica
print("Score del modelo de Regresion Logistica:")
display(cross_val_score(logistic_regression_model, X_cat, y_cat, cv=k_folds))

# Modelo de Arbol de Decision
decision_tree_model = DecisionTreeClassifier()
print("Score del modelo de Arbol de Decision:")
display(cross_val_score(decision_tree_model, X_cat, y_cat, cv=k_folds))

# Debido a que el modelo de Arbol de Decision tiene un score mas alto que el modelo de Regresion Logistica, elegimos el modelo de Arbol de Decision

# Definimos los hiperparametros para el modelo de Arbol de Decision
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Realizamos la busqueda de hiperparametros con GridSearchCV y 5 folds de validacion cruzada (k_folds) y scoring de MSE (mean_squared_error) para el Arbol de Decision sin afinar hiperparametros (DecisionTreeRegressor()) y los hiperparametros definidos en el diccionario (param_grid)

grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv = k_folds, scoring = make_scorer(accuracy_score))
grid_search.fit(X_cat, y_cat)

# Evaluamos el Arbol de Decision con hiperparametros optimizados
dt_optimized = DecisionTreeClassifier(**grid_search.best_params_)
dt_optimized_scores = cross_val_score(dt_optimized, X_cat, y_cat, cv = k_folds, scoring = make_scorer(accuracy_score))

# Calculamos la media y desviacion estandar de los puntajes de MSE para el Arbol de Decision optimizado
print("Arbol de Decision optimizado MSE:", round(dt_optimized_scores.mean(), 2))


#### Respondiendo a las preguntas:

##### ¿Que formas tienen de evaluar los resultados de cada árbol de decisión?
Las formas mas usadas que son "Logistic Regression" y "Decision Tree Classifier", usan la "Acuracy" para evaluar los resultados con un "Score".
##### ¿Como eligen "el mejor árbol"?
El mejor arbol es el que tiene el mejor score, es decir el que mejor se comporta con los datos de testeo. (Igual que en el punto 5)
##### ¿Como se comparan los resultados de los modelos de regresión lineal y de árbol de decisión?
Al igual que en el punto 5, se comparan viendo el score, el que tenga un score mas alto es el que mejor se comporta con los datos de testeo.


### Ej 7. 


Comparar distintos métodos de validación cruzada. ¿Que ventajas y desventajas tiene cada uno?




### Ej 8. 


Escribir una conclusión sobre el trabajo realizado.




### Bonus 1.


Investigar los métodos GridSearch y RandomSearch para la búsqueda de hiperparámetros. Utilizarlos para encontrar los mejores hiperparámetros para los modelos.



### Bonus 2.


Para ya sea el atributo categórico como para el numérico, elegir otro modelo de clasificación o regresión que no haya sido utilizado anteriormente. Entrenar el modelo y comparar los resultados con los obtenidos anteriormente.



### Entrega y Bibliografia



#### Formato de entrega

El trabajo práctico se debe realizar en un notebook de Jupyter. El notebook debe estar subido a un repositorio de GitHub. El link al repositorio debe ser completado en el siguiente [Google Form]().


#### Fecha de entrega

El trabajo práctico se debe entregar hasta el 17/09/2023 a las 23:59hs.

#### Bibliografía

- [SciKit-Learn](https://scikit-learn.org/stable/)
- [Seaborn](https://seaborn.pydata.org/)
- [Kaggle](https://www.kaggle.com/)
- [Cross Validation](https://scikit-learn.org/stable/modules/cross_validation.html)